# IXS/ETH Vault Performance Analysis

The UniV4 pool liquidity is primarily managed by an Arrakis vault (`0x90bde935ce7feb6636afd5a1a0340af45eeae600`).

### Questions

1. How have vault token amounts (IXS and ETH) and vault composition (USD value split) evolved over time?
2. How does vault performance compare to holding the initial amounts (HODL) over time since inception?
3. What is the theoretical full-range LP performance over the same period?
4. What is the performance differential, and what are the tradeoffs of active concentrated management vs passive LP?

In [ ]:
import os
import sys
import math
from web3 import Web3
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

sys.path.append("..")

from src.config import (
    STATE_VIEW, IXS, ETH,
    IXS_POOL_ID_BYTES, ARRAKIS_VAULT, CHAINLINK_ETH_USD,
)
from src.abis import STATEVIEW_ABI_EXTENDED, ARRAKIS_VAULT_ABI, CHAINLINK_ABI
from src.block_utils import (
    generate_daily_block_samples, blocks_to_timestamps,
    timestamps_to_dates, get_latest_block,
)
from src.price_feeds import get_eth_usd_at_block, get_ixs_eth_price_from_v4
from src.vault_performance import (
    batch_vault_underlying,
    calculate_hodl_value, calculate_fullrange_lp_value,
    compute_annualized_return, decompose_vault_returns,
)
from src.migration_detection import find_v4_pool_creation_block

load_dotenv()
os.makedirs('plots', exist_ok=True)
os.makedirs('data', exist_ok=True)
w3 = Web3(Web3.HTTPProvider(os.getenv('rpc_url_mainnet')))
print(f'Connected: {w3.is_connected()}')
print(f'Latest block: {w3.eth.block_number}')

In [ ]:
stateview = w3.eth.contract(address=Web3.to_checksum_address(STATE_VIEW), abi=STATEVIEW_ABI_EXTENDED)
vault = w3.eth.contract(address=Web3.to_checksum_address(ARRAKIS_VAULT), abi=ARRAKIS_VAULT_ABI)
chainlink = w3.eth.contract(address=Web3.to_checksum_address(CHAINLINK_ETH_USD), abi=CHAINLINK_ABI)

# Vault token0 = IXS, token1 = native ETH
vt0 = vault.functions.token0().call()
vt1 = vault.functions.token1().call()
ixs_is_token0_vault = vt0.lower() == IXS.lower()
print(f"Vault token0: {vt0} ({'IXS' if ixs_is_token0_vault else 'ETH'})")
print(f"Vault token1: {vt1} ({'ETH' if ixs_is_token0_vault else 'IXS'})")

## Theory: LP Benchmarks and Return Decomposition

### HODL Benchmark

Hold the initial token amounts without providing liquidity:

$$V_{\text{HODL}}(t) = n_0 \cdot P_0(t) + n_1 \cdot P_1(t)$$

where $n_0, n_1$ are initial token amounts and $P_0(t), P_1(t)$ are current USD prices.

### Full-Range LP (Impermanent Loss)

A full-range LP (V2-style, $x \cdot y = k$) suffers impermanent loss as prices diverge. Define:

$$r = \frac{p_t}{p_0}$$

where $p_0$ is the initial price ratio and $p_t$ is the current price ratio. As price moves, the pool rebalances reserves:

$$x_t = x_0 / \sqrt{r}, \quad y_t = y_0 \cdot \sqrt{r}$$

The LP value relative to HODL is the **IL factor**:

$$\frac{V_{\text{LP}}}{V_{\text{HODL}}} = \frac{2\sqrt{r}}{1 + r} \leq 1$$

Equality holds only at $r = 1$ (no price change). Example: at $r = 4$ (2x price move), IL factor = 0.8 → 20% loss vs HODL.

### Concentrated IL Amplification

For a position concentrated in range $[p_a, p_b]$, the IL amplification factor is:

$$A = \frac{\sqrt{p_b} - \sqrt{p_a}}{\sqrt{p_b} - \sqrt{p}} \cdot \frac{1}{\sqrt{p_a}}$$

In the limit of a very narrow range ($p_a \to p_b$), this approaches leverage-like behavior. A narrower range means:
- **More fees per dollar of TVL** — all capital participates in every trade within range
- **More IL per price move** — position behaves like a levered full-range LP

This is the fundamental tradeoff Arrakis manages: tighter ranges earn more fees but risk more IL.

### Return Decomposition

We decompose vault returns into three additive components:

$$V_{\text{vault}} = V_{\text{HODL}} + \underbrace{(V_{\text{fullrange}} - V_{\text{HODL}})}_{\text{IL (full-range)}} + \underbrace{(V_{\text{vault}} - V_{\text{fullrange}})}_{\text{management premium}}$$

- **Price return** = $V_{\text{HODL}}(t) - V_{\text{HODL}}(0)$ — pure token price movement
- **IL (full-range)** = $V_{\text{fullrange}} - V_{\text{HODL}}$ — cost of providing full-range liquidity (always $\leq 0$)
- **Management premium** = $V_{\text{vault}} - V_{\text{fullrange}}$ — Arrakis's total value-add (fee revenue + rebalancing alpha)

This identity is exact and verifiable: initial value + price return + IL + management premium = vault value.

---

## Setup: Vault Start & Daily Sampling

In [ ]:
latest_block = get_latest_block(w3)
search_start = latest_block - 7200 * 365

# Use V4 pool creation as start (need pool prices to exist)
print("Finding V4 pool creation block...")
v4_start = find_v4_pool_creation_block(
    stateview, IXS_POOL_ID_BYTES, search_start, latest_block
)
print(f"V4 start: block {v4_start:,}")

sample_blocks = generate_daily_block_samples(v4_start, latest_block)
print(f"Daily samples: {len(sample_blocks)}")

print("Fetching timestamps...")
sample_ts = blocks_to_timestamps(w3, sample_blocks)
sample_dates = timestamps_to_dates(sample_ts)
dates = pd.to_datetime(sample_dates)
print(f"Date range: {sample_dates[0]} to {sample_dates[-1]}")

---

## Q1. Vault Token Amounts and Composition Over Time

In [ ]:
print("Fetching vault underlying at each block...")
vault_data = batch_vault_underlying(vault, sample_blocks)

# Parse: vault token0=IXS, token1=ETH
if ixs_is_token0_vault:
    ixs_amounts = [d[1] / 1e18 for d in vault_data]
    eth_amounts = [d[2] / 1e18 for d in vault_data]
else:
    ixs_amounts = [d[2] / 1e18 for d in vault_data]
    eth_amounts = [d[1] / 1e18 for d in vault_data]

print(f"Initial: {ixs_amounts[0]:,.2f} IXS, {eth_amounts[0]:.4f} ETH")
print(f"Current: {ixs_amounts[-1]:,.2f} IXS, {eth_amounts[-1]:.4f} ETH")

In [ ]:
fig, ax1 = plt.subplots(figsize=(14, 6))

ax1.set_xlabel('Date')
ax1.set_ylabel('IXS Amount', color='tab:blue')
ax1.plot(dates, ixs_amounts, color='tab:blue', alpha=0.8, linewidth=1.5, label='IXS')
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax1.tick_params(axis='x', rotation=45)

ax2 = ax1.twinx()
ax2.set_ylabel('ETH Amount', color='tab:orange')
ax2.plot(dates, eth_amounts, color='tab:orange', alpha=0.8, linewidth=1.5, label='ETH')
ax2.tick_params(axis='y', labelcolor='tab:orange')

fig.suptitle('Arrakis Vault: Token Amounts Over Time', fontsize=14, fontweight='bold')
fig.legend(loc='upper left', bbox_to_anchor=(0.12, 0.88))
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('plots/ixs_vault_token_amounts.png', dpi=150, bbox_inches='tight')
plt.show()

### Price Data

- ETH/USD from Chainlink oracle
- IXS/ETH from V4 pool sqrtPriceX96
- IXS/USD derived = IXS/ETH × ETH/USD

In [ ]:
from src.price_feeds import batch_eth_usd_prices

print("Fetching ETH/USD from Chainlink...")
eth_usd_prices = batch_eth_usd_prices(chainlink, sample_blocks)

print("Fetching IXS/ETH from V4 pool...")
ixs_eth_prices = []
for block in sample_blocks:
    try:
        # ETH is currency0 (address(0)), IXS is currency1
        # sqrtPriceX96 gives IXS/ETH (token1 per token0)
        # We want ETH per IXS for pricing
        price = get_ixs_eth_price_from_v4(
            stateview, IXS_POOL_ID_BYTES, block, ixs_is_currency0=False
        )
        ixs_eth_prices.append(price)
    except Exception:
        ixs_eth_prices.append(0.0)

# IXS/USD = (ETH per IXS) × ETH/USD
ixs_usd_prices = [ie * eu for ie, eu in zip(ixs_eth_prices, eth_usd_prices)]

print(f"\nETH/USD range: ${min(eth_usd_prices):,.2f} - ${max(eth_usd_prices):,.2f}")
nz = [p for p in ixs_usd_prices if p > 0]
if nz:
    print(f"IXS/USD range: ${min(nz):.6f} - ${max(nz):.6f}")

### Vault Composition (USD)

In [ ]:
ixs_usd_values = [amt * price for amt, price in zip(ixs_amounts, ixs_usd_prices)]
eth_usd_values = [amt * price for amt, price in zip(eth_amounts, eth_usd_prices)]
total_usd = [i + e for i, e in zip(ixs_usd_values, eth_usd_values)]

fig, ax = plt.subplots(figsize=(14, 6))
ax.stackplot(dates, ixs_usd_values, eth_usd_values,
             labels=['IXS (USD)', 'ETH (USD)'],
             colors=['#4e79a7', '#f28e2b'], alpha=0.8)
ax.plot(dates, total_usd, 'k-', linewidth=1.5, alpha=0.5, label='Total')

ax.set_xlabel('Date')
ax.set_ylabel('Value (USD)')
ax.set_title('Arrakis Vault: Composition Over Time', fontsize=14, fontweight='bold')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.savefig('plots/ixs_vault_composition.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Initial value: ${total_usd[0]:,.2f}")
print(f"Current value: ${total_usd[-1]:,.2f}")

---

## Q2. Vault Performance vs HODL Benchmark

HODL benchmark: hold the initial token amounts without providing liquidity.

$$V_{\text{HODL}}(t) = n_0 \cdot P_0(t) + n_1 \cdot P_1(t)$$

In [ ]:
init_ixs = ixs_amounts[0]
init_eth = eth_amounts[0]

hodl_values = [
    calculate_hodl_value(init_ixs, init_eth, iu, eu)
    for iu, eu in zip(ixs_usd_prices, eth_usd_prices)
]

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(dates, total_usd, 'g-', linewidth=2, label='Arrakis Vault')
ax.plot(dates, hodl_values, 'b--', linewidth=2, label='HODL')

ax.set_xlabel('Date')
ax.set_ylabel('Value (USD)')
ax.set_title('Vault vs HODL', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.savefig('plots/ixs_vault_vs_hodl.png', dpi=150, bbox_inches='tight')
plt.show()

if hodl_values[-1] > 0:
    print(f"Vault: ${total_usd[-1]:,.2f}")
    print(f"HODL:  ${hodl_values[-1]:,.2f}")
    print(f"Diff:  {(total_usd[-1]/hodl_values[-1] - 1)*100:+.2f}%")

---

## Q3. Theoretical Full-Range LP Performance

Full-range LP (V2-style) with impermanent loss:

$$V_{\text{LP}} = V_{\text{HODL}} \times \frac{2\sqrt{r}}{1 + r}$$

where $r = p_t / p_0$ is the relative price ratio change.

In [ ]:
init_ixs_usd = ixs_usd_prices[0]
init_eth_usd = eth_usd_prices[0]

fullrange_values = [
    calculate_fullrange_lp_value(
        init_ixs, init_eth, init_ixs_usd, init_eth_usd, iu, eu
    )
    for iu, eu in zip(ixs_usd_prices, eth_usd_prices)
]

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(dates, total_usd, 'g-', linewidth=2, label='Arrakis Vault (Concentrated)')
ax.plot(dates, hodl_values, 'b--', linewidth=2, label='HODL')
ax.plot(dates, fullrange_values, 'r:', linewidth=2, label='Full-Range LP (V2-style)')

ax.set_xlabel('Date')
ax.set_ylabel('Value (USD)')
ax.set_title('Vault vs HODL vs Full-Range LP', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.savefig('plots/ixs_vault_performance_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

---

## Q4. Performance Differential and Tradeoffs

In [ ]:
print("=" * 60)
print("VAULT PERFORMANCE SUMMARY")
print("=" * 60)
print(f"Period: {sample_dates[0]} to {sample_dates[-1]}")
print(f"\nInitial: {init_ixs:,.2f} IXS + {init_eth:.4f} ETH = ${total_usd[0]:,.2f}")

print(f"\nFinal values:")
print(f"  Vault:      ${total_usd[-1]:,.2f}")
print(f"  HODL:       ${hodl_values[-1]:,.2f}")
print(f"  Full-Range: ${fullrange_values[-1]:,.2f}")

if total_usd[0] > 0:
    print(f"\nReturns vs initial:")
    print(f"  Vault:      {(total_usd[-1]/total_usd[0] - 1)*100:+.2f}%")
    print(f"  HODL:       {(hodl_values[-1]/total_usd[0] - 1)*100:+.2f}%")
    print(f"  Full-Range: {(fullrange_values[-1]/total_usd[0] - 1)*100:+.2f}%")

if hodl_values[-1] > 0:
    il_fr = (fullrange_values[-1]/hodl_values[-1] - 1) * 100
    il_vault = (total_usd[-1]/hodl_values[-1] - 1) * 100
    print(f"\nImpermanent Loss (vs HODL):")
    print(f"  Full-Range: {il_fr:+.2f}%")
    print(f"  Vault:      {il_vault:+.2f}%")
    print(f"\nThe vault {'outperforms' if il_vault > il_fr else 'underperforms'} full-range LP by {abs(il_vault - il_fr):.2f}pp")

### Return Decomposition

$$V_{\text{vault}} = V_{\text{HODL}} + \underbrace{(V_{\text{fullrange}} - V_{\text{HODL}})}_{\text{IL (full-range)}} + \underbrace{(V_{\text{vault}} - V_{\text{fullrange}})}_{\text{management premium}}$$

In [ ]:
# Decompose vault returns
decomp = decompose_vault_returns(total_usd, hodl_values, fullrange_values)

price_returns = [d['price_return'] for d in decomp]
il_components = [d['il_fullrange'] for d in decomp]
mgmt_premiums = [d['management_premium'] for d in decomp]

# Stacked area chart
fig, ax = plt.subplots(figsize=(14, 7))

# Plot as changes from initial value
initial_val = hodl_values[0]
ax.fill_between(dates, 0, price_returns, alpha=0.4, color='steelblue', label='Price Return')
ax.fill_between(dates, price_returns,
                [p + il for p, il in zip(price_returns, il_components)],
                alpha=0.4, color='salmon', label='IL (Full-Range)')
ax.fill_between(dates, [p + il for p, il in zip(price_returns, il_components)],
                [p + il + m for p, il, m in zip(price_returns, il_components, mgmt_premiums)],
                alpha=0.4, color='mediumseagreen', label='Management Premium')

# Vault total change line
vault_changes = [v - initial_val for v in total_usd]
ax.plot(dates, vault_changes, 'k-', linewidth=1.5, alpha=0.7, label='Vault Total Change')

ax.set_xlabel('Date')
ax.set_ylabel('USD Change from Initial')
ax.set_title('Vault Return Decomposition', fontsize=14, fontweight='bold')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='black', linewidth=0.5, alpha=0.5)
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.savefig('plots/ixs_vault_decomposition.png', dpi=150, bbox_inches='tight')
plt.show()

# Print final decomposition
d_final = decomp[-1]
print(f"Final Return Decomposition (USD):")
print(f"  Initial value:       ${initial_val:,.2f}")
print(f"  + Price return:      ${d_final['price_return']:+,.2f}")
print(f"  + IL (full-range):   ${d_final['il_fullrange']:+,.2f}")
print(f"  + Mgmt premium:      ${d_final['management_premium']:+,.2f}")
print(f"  = Vault value:       ${total_usd[-1]:,.2f}")

# Verify identity
reconstructed = initial_val + d_final['price_return'] + d_final['il_fullrange'] + d_final['management_premium']
print(f"\n  Identity check: {initial_val:.2f} + {d_final['price_return']:.2f} + {d_final['il_fullrange']:.2f} + {d_final['management_premium']:.2f} = {reconstructed:.2f} (vault = {total_usd[-1]:.2f})")

In [ ]:
# Waterfall bar chart: initial → +price → +IL → +premium → vault
labels = ['Initial', '+ Price\nReturn', '+ IL\n(Full-Range)', '+ Mgmt\nPremium', 'Vault\nFinal']
values = [
    initial_val,
    d_final['price_return'],
    d_final['il_fullrange'],
    d_final['management_premium'],
    total_usd[-1],
]

# For waterfall: compute cumulative bottoms
bottoms = [0, initial_val, initial_val + d_final['price_return'],
           initial_val + d_final['price_return'] + d_final['il_fullrange'], 0]
bar_heights = values.copy()
colors = ['#4e79a7', '#59a14f' if values[1] >= 0 else '#e15759',
          '#e15759' if values[2] < 0 else '#59a14f',
          '#59a14f' if values[3] >= 0 else '#e15759', '#4e79a7']

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(labels, bar_heights, bottom=bottoms, color=colors, alpha=0.85, edgecolor='white', linewidth=1.5)

# Add value labels
for bar, val, bottom in zip(bars, values, bottoms):
    y_pos = bottom + val / 2 if val > 0 else bottom + val / 2
    label = f'${val:+,.0f}' if val != initial_val and val != total_usd[-1] else f'${val:,.0f}'
    ax.text(bar.get_x() + bar.get_width()/2, y_pos, label,
            ha='center', va='center', fontweight='bold', fontsize=10, color='white')

# Connect bars with lines
for i in range(len(labels) - 2):
    top = bottoms[i] + values[i]
    ax.plot([i + 0.4, i + 0.6], [top, top], 'k-', linewidth=0.8, alpha=0.5)

ax.set_ylabel('USD Value')
ax.set_title('Return Attribution Waterfall', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.2, axis='y')
plt.tight_layout()
plt.savefig('plots/ixs_vault_waterfall.png', dpi=150, bbox_inches='tight')
plt.show()

### Annualized Performance

In [ ]:
# Compute observation period in days
from datetime import datetime
start_dt = datetime.strptime(sample_dates[0], '%Y-%m-%d')
end_dt = datetime.strptime(sample_dates[-1], '%Y-%m-%d')
days = (end_dt - start_dt).days
print(f"Observation period: {days} days ({days/30.44:.1f} months)")

# Annualized returns
vault_ann = compute_annualized_return(total_usd[0], total_usd[-1], days)
hodl_ann = compute_annualized_return(hodl_values[0], hodl_values[-1], days)
fr_ann = compute_annualized_return(fullrange_values[0], fullrange_values[-1], days)

print(f"\n{'='*50}")
print(f"  ANNUALIZED PERFORMANCE")
print(f"{'='*50}")
print(f"{'Strategy':<20} {'Total Return':>14} {'Annualized':>12}")
print("-" * 46)

vault_ret = (total_usd[-1] / total_usd[0] - 1) * 100
hodl_ret = (hodl_values[-1] / hodl_values[0] - 1) * 100
fr_ret = (fullrange_values[-1] / fullrange_values[0] - 1) * 100

print(f"{'Arrakis Vault':<20} {vault_ret:>+13.2f}% {vault_ann:>+11.2f}%")
print(f"{'HODL':<20} {hodl_ret:>+13.2f}% {hodl_ann:>+11.2f}%")
print(f"{'Full-Range LP':<20} {fr_ret:>+13.2f}% {fr_ann:>+11.2f}%")

# Management premium in pp (annualized)
mgmt_pp = vault_ann - fr_ann
print(f"\nManagement premium (vs full-range): {mgmt_pp:+.2f}pp annualized")

# Component percentages
if total_usd[-1] != initial_val:
    total_change = total_usd[-1] - initial_val
    print(f"\nReturn Attribution (% of total change):")
    print(f"  Price return:      {d_final['price_return']/total_change*100:+.1f}%")
    print(f"  IL (full-range):   {d_final['il_fullrange']/total_change*100:+.1f}%")
    print(f"  Mgmt premium:      {d_final['management_premium']/total_change*100:+.1f}%")

---

## Additional Analysis: Active Liquidity Management

How does Arrakis create value? By actively rebalancing the vault's tick ranges to keep liquidity concentrated near the current price. We track ranges via the Arrakis module contract's `getRanges()` function, detect rebalance events, and compute capital efficiency.

In [ ]:
from src.config import ARRAKIS_MODULE
from src.abis import ARRAKIS_MODULE_ABI
from src.vault_rebalancing import (
    batch_pool_state_and_ranges, detect_rebalances,
    ranges_to_prices, compute_capital_efficiency,
)

module = w3.eth.contract(
    address=Web3.to_checksum_address(ARRAKIS_MODULE), abi=ARRAKIS_MODULE_ABI
)

print("Fetching pool state + vault ranges at each sample block...")
range_data = batch_pool_state_and_ranges(
    stateview, module, IXS_POOL_ID_BYTES, sample_blocks
)
print(f"Fetched {len(range_data)} samples")

# Detect rebalances
rebalance_idx = detect_rebalances(range_data)
print(f"\nDetected {len(rebalance_idx)} rebalance events")
for idx in rebalance_idx:
    prev = range_data[idx - 1]["ranges"]
    curr = range_data[idx]["ranges"]
    print(f"  Block {range_data[idx]['block']:,} ({sample_dates[idx]}): {prev} -> {curr}")

In [ ]:
# Estimate rebalancing gas cost
from src.vault_rebalancing import estimate_rebalancing_gas_cost

# Fetch baseFeePerGas at each rebalance block
rebalance_blocks = [range_data[idx]["block"] for idx in rebalance_idx]
gas_prices_gwei = []
for blk in rebalance_blocks:
    block_data = w3.eth.get_block(blk)
    base_fee = block_data.get("baseFeePerGas", 0)
    gas_prices_gwei.append(base_fee / 1e9)  # wei → gwei

avg_gas_price = sum(gas_prices_gwei) / len(gas_prices_gwei) if gas_prices_gwei else 0
avg_eth_usd = sum(eth_usd_prices) / len(eth_usd_prices)

# Arrakis rebalance() typically costs 200K-400K gas
AVG_GAS_PER_REBALANCE = 300_000
gas_cost_usd = estimate_rebalancing_gas_cost(
    len(rebalance_idx), AVG_GAS_PER_REBALANCE, avg_gas_price, avg_eth_usd
)

print(f"Rebalancing Gas Cost Estimate")
print(f"  Rebalances:          {len(rebalance_idx)}")
print(f"  Avg gas price:       {avg_gas_price:.1f} gwei")
print(f"  Avg ETH/USD:         ${avg_eth_usd:,.0f}")
print(f"  Est. gas per rebal:  {AVG_GAS_PER_REBALANCE:,}")
print(f"  Total gas cost:      ${gas_cost_usd:,.2f}")
if d_final['management_premium'] != 0:
    print(f"  As % of mgmt prem:   {gas_cost_usd / abs(d_final['management_premium']) * 100:.1f}%")

In [ ]:
# Plot: Price + Range Bands + Rebalance Markers
# V4 pool: currency0=ETH, currency1=IXS → sqrtPriceX96 encodes IXS/ETH
# We want ETH per IXS for the y-axis (same units as ixs_eth_prices)

fig, ax = plt.subplots(figsize=(14, 7))

# Pool price over time (ETH per IXS = 1/price since price = IXS/ETH)
pool_prices_eth_per_ixs = []
for rd in range_data:
    if rd["price"] > 0:
        pool_prices_eth_per_ixs.append(1.0 / rd["price"])
    else:
        pool_prices_eth_per_ixs.append(0.0)

ax.plot(dates, pool_prices_eth_per_ixs, color='#2ca02c', linewidth=1.8, label='IXS/ETH Price', zorder=3)

# Shade vault range bands (convert ticks to ETH-per-IXS prices)
for i in range(len(range_data)):
    rd = range_data[i]
    if not rd["ranges"]:
        continue
    # Date span for this sample
    d_start = dates[i]
    d_end = dates[i + 1] if i + 1 < len(dates) else dates[i]

    for tick_lower, tick_upper in rd["ranges"]:
        # tick_to_sqrt_price^2 gives IXS/ETH → invert for ETH/IXS
        price_lower_ixs_eth = tick_to_sqrt_price(tick_lower) ** 2
        price_upper_ixs_eth = tick_to_sqrt_price(tick_upper) ** 2
        # Invert: ETH per IXS
        band_low = 1.0 / price_upper_ixs_eth if price_upper_ixs_eth > 0 else 0
        band_high = 1.0 / price_lower_ixs_eth if price_lower_ixs_eth > 0 else 0
        ax.axhspan(band_low, band_high, xmin=i/len(dates), xmax=(i+1)/len(dates),
                   alpha=0.15, color='#2ca02c', zorder=1)

# Rebalance markers
for idx in rebalance_idx:
    ax.axvline(dates[idx], color='red', linestyle='--', alpha=0.6, linewidth=1, zorder=2)
if rebalance_idx:
    ax.axvline(dates[rebalance_idx[0]], color='red', linestyle='--', alpha=0.6,
               linewidth=1, label='Rebalance Event')

ax.set_xlabel('Date')
ax.set_ylabel('ETH per IXS')
ax.set_title('Price Tracking: Vault Range Bands Follow the Price', fontsize=14, fontweight='bold')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.savefig('plots/ixs_vault_price_ranges.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Plot: Active Liquidity Over Time
liq_values = [rd["liquidity"] for rd in range_data]

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(dates, liq_values, color='#4e79a7', linewidth=1.8, label='Active In-Range Liquidity')

for idx in rebalance_idx:
    ax.axvline(dates[idx], color='red', linestyle='--', alpha=0.5, linewidth=1)
if rebalance_idx:
    ax.axvline(dates[rebalance_idx[0]], color='red', linestyle='--', alpha=0.5,
               linewidth=1, label='Rebalance Event')

ax.set_xlabel('Date')
ax.set_ylabel('Liquidity (L)')
ax.set_title('Active In-Range Liquidity Over Time', fontsize=14, fontweight='bold')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.savefig('plots/ixs_vault_active_liquidity.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Liquidity range: {min(liq_values):,.0f} - {max(liq_values):,.0f}")
print(f"Current liquidity: {liq_values[-1]:,.0f}")

In [ ]:
# Plot: Capital Efficiency Over Time
# CE = actual L / theoretical full-range L for same TVL
# Must use raw (wei) amounts since Uniswap L = sqrt(x_raw * y_raw)
if ixs_is_token0_vault:
    raw_ixs = [d[1] for d in vault_data]
    raw_eth = [d[2] for d in vault_data]
else:
    raw_ixs = [d[2] for d in vault_data]
    raw_eth = [d[1] for d in vault_data]

ce_ratios = []
for i, rd in enumerate(range_data):
    ce = compute_capital_efficiency(rd["liquidity"], raw_ixs[i], raw_eth[i])
    ce_ratios.append(ce)

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(dates, ce_ratios, color='#59a14f', linewidth=1.8, label='Capital Efficiency Ratio')
ax.axhline(y=1.0, color='gray', linestyle=':', linewidth=1.5, label='Full-Range Baseline (1.0x)')

for idx in rebalance_idx:
    ax.axvline(dates[idx], color='red', linestyle='--', alpha=0.5, linewidth=1)
if rebalance_idx:
    ax.axvline(dates[rebalance_idx[0]], color='red', linestyle='--', alpha=0.5,
               linewidth=1, label='Rebalance Event')

ax.set_xlabel('Date')
ax.set_ylabel('CE Ratio (Concentrated / Full-Range)')
ax.set_title('Capital Efficiency: Concentration Multiplier Over Time', fontsize=14, fontweight='bold')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.savefig('plots/ixs_vault_capital_efficiency.png', dpi=150, bbox_inches='tight')
plt.show()

nz_ce = [c for c in ce_ratios if c > 0]
if nz_ce:
    print(f"Capital efficiency: {min(nz_ce):.1f}x - {max(nz_ce):.1f}x (median: {sorted(nz_ce)[len(nz_ce)//2]:.1f}x)")
    print(f"Current: {ce_ratios[-1]:.1f}x more efficient than full-range")

In [ ]:
# Arrakis-Style Snapshot: 2x2 grid showing range evolution over time
# X = price (ETH per IXS), Y = liquidity depth (narrower = taller)
from src.amm_math import tick_to_sqrt_price
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

def draw_arrakis_snapshot(ax, snap, label, date_label):
    """Draw one Arrakis-style liquidity concentration snapshot."""
    current_eth_per_ixs = 1.0 / snap["price"] if snap["price"] > 0 else 0

    # Convert tick ranges to ETH-per-IXS prices
    range_bars = []
    for tl, tu in snap["ranges"]:
        p_lo_ixs = tick_to_sqrt_price(tl) ** 2
        p_hi_ixs = tick_to_sqrt_price(tu) ** 2
        eth_lo = 1.0 / p_hi_ixs
        eth_hi = 1.0 / p_lo_ixs
        range_bars.append((eth_lo, eth_hi, tu - tl))

    all_eth = [current_eth_per_ixs] + [v for lo, hi, _ in range_bars for v in (lo, hi)]
    x_min, x_max = min(all_eth) * 0.5, max(all_eth) * 1.15

    # Full-range bar: wide, flat, blue
    fr_h = 0.6
    ax.bar((x_min + x_max) / 2, fr_h, width=(x_max - x_min) * 0.98,
           bottom=0, color='#a8d5e2', alpha=0.5, edgecolor='#5b9bd5', linewidth=1, zorder=1)
    ax.text((x_min + x_max) / 2, fr_h / 2, 'Full Range',
            fontsize=7, ha='center', va='center', color='#3a7ca5', fontweight='bold', alpha=0.7)
    ax.annotate('', xy=(x_min * 1.01, fr_h / 2), xytext=(x_min * 1.08, fr_h / 2),
                arrowprops=dict(arrowstyle='<-', color='#5b9bd5', lw=1.2))
    ax.annotate('', xy=(x_max * 0.99, fr_h / 2), xytext=(x_max * 0.93, fr_h / 2),
                arrowprops=dict(arrowstyle='<-', color='#5b9bd5', lw=1.2))

    # Vault ranges: staircase (widest first, narrowest tallest)
    range_bars.sort(key=lambda r: -(r[1] - r[0]))
    n = len(range_bars)
    for i, (eth_lo, eth_hi, tw) in enumerate(range_bars):
        height = 1.5 + 2.5 * (i + 1) / n
        ax.bar((eth_lo + eth_hi) / 2, height, width=eth_hi - eth_lo,
               bottom=fr_h, color='#f5c896', alpha=0.8, edgecolor='#d4915e',
               linewidth=1.5, zorder=2 + i)

    # Current price
    ax.axvline(current_eth_per_ixs, color='#333333', linestyle=':', linewidth=2.5, zorder=10)

    # Token labels
    ax.text(x_min + (current_eth_per_ixs - x_min) * 0.3, 5.8, 'ETH',
            fontsize=14, fontweight='bold', color='#5b9bd5', ha='center', style='italic')
    ax.text(current_eth_per_ixs + (x_max - current_eth_per_ixs) * 0.7, 5.8, 'IXS',
            fontsize=14, fontweight='bold', color='#d4915e', ha='center', style='italic')

    # Bootstrapping label
    narrowest = min(range_bars, key=lambda r: r[1] - r[0])
    ax.text((narrowest[0] + narrowest[1]) / 2, fr_h + 1.5 + 2.5 + 0.3,
            'Bootstrapping\nRanges', fontsize=7, ha='center', va='bottom',
            style='italic', color='#8b6914')

    ax.set_xlim(x_min, x_max)
    ax.set_ylim(-0.15, 6.8)
    ax.set_xlabel('Price', fontsize=10)
    ax.set_title(f'{label} ({date_label})', fontsize=11, fontweight='bold')
    ax.set_yticks([])
    ax.set_xticklabels([])
    for sp in ['top', 'right', 'left']:
        ax.spines[sp].set_visible(False)
    ax.annotate('', xy=(x_max, 0), xytext=(x_min, 0),
                arrowprops=dict(arrowstyle='->', color='gray', lw=1.2))

# Pick 4 representative time points
# First with ranges, ~1/3, ~2/3, and latest
first_with_ranges = next(i for i, rd in enumerate(range_data) if rd["ranges"])
n_samples = len(range_data)
snap_indices = [
    first_with_ranges,
    first_with_ranges + (n_samples - first_with_ranges) // 3,
    first_with_ranges + 2 * (n_samples - first_with_ranges) // 3,
    -1,
]

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
labels = ['Launch', 'Mid-January', 'Early February', 'Current']
for ax, idx, label in zip(axes.flat, snap_indices, labels):
    draw_arrakis_snapshot(ax, range_data[idx], label, sample_dates[idx])

legend_elements = [
    Patch(facecolor='#a8d5e2', alpha=0.5, edgecolor='#5b9bd5', label='Full Range'),
    Patch(facecolor='#f5c896', alpha=0.8, edgecolor='#d4915e', label='Vault Ranges'),
    Line2D([0], [0], color='#333333', linestyle=':', linewidth=2.5, label='Current Price'),
]
fig.legend(handles=legend_elements, loc='lower center', ncol=3, fontsize=11,
           framealpha=0.9, bbox_to_anchor=(0.5, -0.01))
fig.suptitle('Liquidity Concentration Over Time: Arrakis Vault Range Evolution',
             fontsize=15, fontweight='bold')
plt.tight_layout(rect=[0, 0.03, 1, 0.96])
plt.savefig('plots/ixs_vault_snapshot_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

---

## Data Export

In [ ]:
# Enhanced summary with annualized figures and decomposition
print("=" * 60)
print("VAULT PERFORMANCE SUMMARY")
print("=" * 60)
print(f"Period: {sample_dates[0]} to {sample_dates[-1]} ({days} days)")
print(f"\nInitial: {init_ixs:,.2f} IXS + {init_eth:.4f} ETH = ${total_usd[0]:,.2f}")

print(f"\n{'Strategy':<20} {'Final USD':>14} {'Return':>10} {'Annualized':>12}")
print("-" * 56)
print(f"{'Arrakis Vault':<20} ${total_usd[-1]:>12,.2f} {vault_ret:>+9.2f}% {vault_ann:>+11.2f}%")
print(f"{'HODL':<20} ${hodl_values[-1]:>12,.2f} {hodl_ret:>+9.2f}% {hodl_ann:>+11.2f}%")
print(f"{'Full-Range LP':<20} ${fullrange_values[-1]:>12,.2f} {fr_ret:>+9.2f}% {fr_ann:>+11.2f}%")

print(f"\nManagement premium: {mgmt_pp:+.2f}pp annualized")
print(f"Rebalancing gas cost: ${gas_cost_usd:,.2f} ({len(rebalance_idx)} rebalances)")

print(f"\nReturn Decomposition (final):")
print(f"  Price return:      ${d_final['price_return']:+,.2f}")
print(f"  IL (full-range):   ${d_final['il_fullrange']:+,.2f}")
print(f"  Mgmt premium:      ${d_final['management_premium']:+,.2f}")

# Export vault summary CSV for D3
os.makedirs('data', exist_ok=True)
vault_export = pd.DataFrame({
    'metric': ['period_start', 'period_end', 'days',
               'vault_initial_usd', 'vault_final_usd', 'hodl_final_usd', 'fullrange_final_usd',
               'vault_return_pct', 'hodl_return_pct', 'fullrange_return_pct',
               'vault_ann_pct', 'hodl_ann_pct', 'fullrange_ann_pct',
               'price_return_usd', 'il_fullrange_usd', 'mgmt_premium_usd',
               'mgmt_premium_ann_pp',
               'n_rebalances', 'gas_cost_usd'],
    'value': [sample_dates[0], sample_dates[-1], days,
              round(total_usd[0], 2), round(total_usd[-1], 2),
              round(hodl_values[-1], 2), round(fullrange_values[-1], 2),
              round(vault_ret, 4), round(hodl_ret, 4), round(fr_ret, 4),
              round(vault_ann, 4), round(hodl_ann, 4), round(fr_ann, 4),
              round(d_final['price_return'], 2), round(d_final['il_fullrange'], 2),
              round(d_final['management_premium'], 2), round(mgmt_pp, 4),
              len(rebalance_idx), round(gas_cost_usd, 2)],
})
vault_export.to_csv('data/vault_summary.csv', index=False)
print(f"\nExported vault summary to data/vault_summary.csv")